In [95]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, NoSuchWindowException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

Instructions:  
• All questions are compulsory.  
• In each of the questions you have to automate the process. You do not have to click on any button, click any
clickable element, enter keywords in search boxes manually. Each process has to be performed via coding.  
• Q1 and Q2 are connected questions i.e. after attempting Q1 proceed to Q2. Do not write whole code from
beginning for Q2.  
• You may use any web scraping library and tools.  
• The question can be attempted in various ways; the correctness of question depends on the output.  
• If you encounter any Null values during scraping, you may replace it by hyphen.  

Request: 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
# Set up the driver
driver = webdriver.Chrome()

# Load the webpage
driver.get("https://www.amazon.in/")

In [3]:
# Enter user term in the search field and click the search icon
USER_INPUT = input("Enter a your search term: ")
search_field = driver.find_element(By.XPATH, "/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
search_field.send_keys(USER_INPUT)
search_field.send_keys(Keys.RETURN)

Enter a your search term: guitar


Request:
2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [4]:
# scrape URLs of the results

urls = []
for page in range(0, 3):
# Find all of the URLs on the current page
    url_elements = driver.find_elements(By.XPATH, '//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
# Append the URLs to the urls list
    for url_element in url_elements:
        urls.append(url_element.get_attribute("href"))
# Click the next button to go to the next page
    next_button = driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next_button.click()
    time.sleep(4)

In [5]:
# Create lists

brand_name = []
name_product = []
price = []
return_ex = []
ex_delivery = []
avail = []
product_URL = []

In [6]:
# Scrape the data from each of the URLs collected

for url in urls:
    driver.get(url)
    time.sleep(2)
    
    try:
        brand = driver.find_element(By.XPATH, '//a[@id="bylineInfo"]')      
        brand_name.append(brand.text)
    except NoSuchElementException:
        brand_name.append('-')    

    time.sleep(1)

    try:
        name = driver.find_element(By.ID, 'productTitle')      
        name_product.append(name.text)
    except NoSuchElementException:
        name_product.append('-')
        
    time.sleep(1)    
 
    try:
        pric = driver.find_element(By.XPATH, '//span[@class="a-price-whole"]')  
        price.append(pric.text)
    except NoSuchElementException:
        price.append('-')
        
    time.sleep(1)

    try:
        return_e = driver.find_element(By.XPATH, '//a[@class="a-size-small a-color-link a-text-normal"]')  
        return_ex.append(return_e.text)
    except NoSuchElementException:
        return_ex.append('-')

    time.sleep(1)

    try:
        delivery= driver.find_element(By.XPATH, '//span[@class="a-text-bold"]')  
        ex_delivery.append(delivery.text)
    except NoSuchElementException:
        ex_delivery.append('-')

    time.sleep(1)
     
    try:
        avai= driver.find_element(By.XPATH, '//span[@class="a-size-medium a-color-success"]')  
        avail.append(avai.text)
    except NoSuchElementException:
        avail.append('-')

In [7]:
product_URL = urls

In [8]:
# Create a dataframe of the scraped data
data = {
    'Brand Name': brand_name,
    'Name of the Product': name_product,
    'Price': price,
    'Return/Exchange': return_ex,
    'Expected Delivery': ex_delivery,
    'Availability': avail,
    'Product URL': product_URL
}

In [9]:
df = pd.DataFrame(data)
df

,Brand Name,Name of the Product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Visit the Kadence Store,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,"6,899",-,"Tuesday, 20 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Brand: Medellin,"Medellin 38"" Acoustic Guitar premium wood, fre...","1,999",-,"Tuesday, 20 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Visit the Kadence Store,Kadence Frontier guitar with Online Guitar lea...,"4,999",-,"Monday, 19 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Visit the Kadence Store,"Kadence rosewood Guitar Frontier Series, Elect...","5,799",-,"Tuesday, 20 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Visit the JUAREZ Store,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","1,994",-,"Monday, 19 June",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
177,Visit the FESTRA Store,Festra R-40- Trussrod Acoustic Guitar Kit with...,"4,270",-,"Wednesday, 21 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
178,Visit the blueberry Store,"Blueberry, VIP-360, 36Inch Acoustic Guitar Kit...","2,999",-,"Wednesday, 21 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
179,Visit the blueberry Store,"Blueberry, VIP-301, 30Inch Acoustic Guitar Kit...","3,290",-,"Wednesday, 21 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
180,Visit the blueberry Store,"Blueberry, VIP-384, 38Inch Cutaway Acoustic Gu...","3,290",-,"Wednesday, 21 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [10]:
# Quit the browser
driver.quit()

Request:
3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [104]:
# Set up the driver
driver = webdriver.Chrome()

In [101]:
# Establish Keywords

keywords = ['fruits', 'cars',  'Machine Learning', 'Guitar', 'Cakes']

In [106]:
#Scrape data
for keyword in keywords:
    # Access Google Images
    driver.get("https://images.google.com/")
    time.sleep(2) 

    # Find the search bar and enter the keyword
    search_bar = driver.find_element(By.NAME, "q")
    search_bar.send_keys(keyword)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2) 

    # Scroll down to load more images
    for _ in range(3): 
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2) 

    # Find the image elements and extract the source URLs
    image_elements = driver.find_elements(By.TAG_NAME, "img")
    image_urls = [element.get_attribute("src") for element in image_elements]
    image_urls = [url for url in image_urls if url and url.startswith("https")]

    # Print the first 10 image URLs
    print(f"Top 10 Images for '{keyword}':")
    for i, url in enumerate(image_urls[:10], 1):
        print(f"{i}. {url}")

Top 10 Images for 'fruits':
1. https://www.google.com/logos/doodles/2023/magdalena-abakanowiczs-93rd-birthday-6753651837109891-6752733080609218-cst.png
2. https://fonts.gstatic.com/s/i/productlogos/lens_2023q2/v2/192px.svg
3. https://www.gstatic.com/ui/v1/menu/light_thumbnail2.png
4. https://www.gstatic.com/ui/v1/menu/dark_thumbnail2.png
5. https://www.gstatic.com/ui/v1/menu/device_default_thumbnail2.png
6. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSlhpkH8nnitw4DBaFdAoFknAhDmb5jDpLf9mTN8c6PNRHCeUUmph92E4M&usqp=CAU
7. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSjgxgR4OAGnxupmqxj5RjgeW7xJNJyCbsFEPvsbdWNjadgevGzLQgH0Nw&usqp=CAU
8. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT7GOpPbKAhDnKKC2KAGL7VEoA0WUfT4ON7QhGkb0sJrbq-gAFPO4lC4Zo&usqp=CAU
9. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR1LngTan9RBQ3JHvGJktONI4lwtmtppmA2bXRHx51vY1zWoPln76RogJE&usqp=CAU
10. https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQJ8oDUyAT-t0tVI_XVQi8FdAAFH2J7Zs8MC1a7

In [107]:
# Quit the browser
driver.quit()

Request:
4. Write a python program to search for a smartphone 'Oneplus Nord' on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [3]:
# Set up the driver
driver = webdriver.Chrome()

# Load the webpage
driver.get("http://www.flipkart.com/")

In [4]:
# click close button

close_button = driver.find_element(By.XPATH, '//button[@class="_2KpZ6l _2doB4z"]')
close_button.click()

In [5]:
# search for Oneplus Nord

search = driver.find_element(By.XPATH, '//input[@class="_3704LK"]')
search.send_keys("Oneplus Nord")
search_button = driver.find_element(By.XPATH, '//button[@class="L0Z3Pu"]')
search_button.click()

In [6]:
# Fetching urls of phones coming on 1st page
flip_urls = []
urls = driver.find_elements(By.XPATH, '//a[@class="_1fQZEK"]')
for url in urls:
    flip_urls.append(url.get_attribute("href"))
    
#Make empty lists   
flip_dict = {}
flip_dict["Brand"] = []
flip_dict["Smartphone"] = []
flip_dict["Color"] = []
flip_dict["RAM"] = []
flip_dict["Storage(ROM)"] = []
flip_dict["Primary Camera"] = []
flip_dict["Secondary Camera"] = []
flip_dict["Display Size"] = []
flip_dict["Battery Capacity"] = []
flip_dict["Price"] = []
flip_dict["URL"] = []

In [7]:
# Scraping data from each url
for url in flip_urls:
    driver.get(url)                      
    flip_dict['URL'].append(url)
    time.sleep(2)
    
    try:
        read_more = driver.find_element(By.XPATH, '//button[@class="_2KpZ6l _1FH0tX"]')     # Button for expanding the specs
        read_more.click()
    except NoSuchElementException:
        print("Exception Occured. Moving to next page")
    
    try:
        brand = driver.find_element(By.XPATH, '//span[@class="B_NuCI"]')      # Extracting Brand from xpath
        flip_dict["Brand"].append(brand.text.split()[0])
    except NoSuchElementException:
        flip_dict['Brand'].append('-')
               
    try:
        name = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')      # Extracting Name from xpath
        flip_dict['Smartphone'].append(name.text)
    except NoSuchElementException:
        flip_dict['Smartphone'].append('-')
    
    try:
        color = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      # Extracting colour from xpath
        flip_dict['Color'].append(color.text)
    except NoSuchElementException:
        flip_dict['Color'].append('-')
    
    try:
        disp_chk = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][2]/div')
        if disp_chk.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  # Extracting Display Size from xpath
        flip_dict['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        flip_dict['Display Size'].append('-')

    try:
        ram = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                # Extracting RAM from xpath
        flip_dict['RAM'].append(ram.text)
    except NoSuchElementException:
        flip_dict['RAM'].append('-')        
        
    try:
        rom = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')         # Extracting Storage(ROM) from xpath
        flip_dict['Storage(ROM)'].append(rom.text)
    except NoSuchElementException:
        flip_dict['Storage(ROM)'].append('-')
    
    try:                                                                                  
        pri_cam = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')     # Extracting Camera from xpath
        flip_dict['Primary Camera'].append(pri_cam.text)
    except NoSuchElementException:
        flip_dict['Primary Camera'].append('-')
    
    try:                                                                                    # Extracting Secondary Camera from xpath
        cam_chk = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if cam_chk != "Secondary Camera" : 
            if driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        flip_dict['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        flip_dict['Secondary Camera'].append('-')
        
    try:
        if driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :   # Extracting Battery Capacity from xpath
            if driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_chk = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :     
                bat_chk = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_chk = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
            bat_cap = driver.find_element(By.XPATH, '//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
        flip_dict['Battery Capacity'].append(bat_cap.text)
    except NoSuchElementException:
        flip_dict['Battery Capacity'].append('-')
    
    try:
        price = driver.find_element(By.XPATH, '//div[@class="_30jeq3 _16Jk6d"]')      # Extracting Price from xpath
        flip_dict['Price'].append(price.text)
    except NoSuchElementException:
        flip_dict['Price'].append('-')

In [8]:
print(len(flip_dict["Brand"]), len(flip_dict["Smartphone"]), len(flip_dict["Price"]), len(flip_dict['URL']))

24 24 24 24


In [9]:
flip_df = pd.DataFrame.from_dict(flip_dict)
flip_df

,Brand,Smartphone,Color,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,OnePlus,Nord CE 2 Lite 5G,Blue Tide,8 GB,128 GB,-,-,16.74 cm (6.59 inch),-,"₹21,974",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,-,-,16.74 cm (6.59 inch),-,"₹18,347",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,-,-,16.74 cm (6.59 inch),-,"₹18,201",https://www.flipkart.com/oneplus-nord-ce-2-lit...
3,OnePlus,Nord 2T 5G,Gray Shadow,8 GB,128 GB,-,-,17.02 cm (6.7 inch),-,"₹27,915",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
4,OnePlus,Nord 2T 5G,Jade Fog,8 GB,128 GB,-,-,17.02 cm (6.7 inch),-,"₹28,119",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
5,OnePlus,Nord 2T 5G,Jade Fog,8 GB,128 GB,-,-,17.02 cm (6.7 inch),-,"₹28,198",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
6,OnePlus,Nord 2T 5G,Gray Shadow,8 GB,128 GB,-,-,17.02 cm (6.7 inch),-,"₹27,788",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
7,OnePlus,Nord,Blue Marble,12 GB,256 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹26,990",https://www.flipkart.com/oneplus-nord-blue-mar...
8,OnePlus,Nord,Gray Onyx,12 GB,256 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹26,990",https://www.flipkart.com/oneplus-nord-gray-ony...
9,OnePlus,Nord,Gray Onyx,8 GB,128 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹24,990",https://www.flipkart.com/oneplus-nord-gray-ony...


In [10]:
# Quit the browser
driver.quit()

Request:
5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [12]:
# Set up the driver
driver = webdriver.Chrome()

# Load the webpage
driver.get("https://maps.google.com/")

In [16]:
search = driver.find_element(By.ID, "searchboxinput")
time.sleep(2)
search.send_keys("New Delhi") 
button = driver.find_element(By.ID, "searchbox-searchbutton")
button.click() 
time.sleep(3)


url_string = driver.current_url
    
cords = re.findall(r'@(.*)data',url_string)
    
if len(cords):
    cordinates = cords[0].split(",")
    if len(cordinates)>=2:
        lat = cordinates[0]
        long = cordinates[1]
    print("Latitude = {}, Longitude = {}".format(lat, long))

Latitude = 28.5275544, Longitude = 77.0441758


In [17]:
# Quit the browser
driver.quit()

Request:
6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [23]:
# Set up the driver
driver = webdriver.Chrome()

# Load the webpage
driver.get("https://www.digit.in/")

In [26]:
#clicking on top 10 option 
top_10=driver.find_element(By.XPATH, "/html/body/div[1]/div/div[4]/ul/li[4]/a")
top_10.click()

time.sleep(3)

#clicking on best gaming laptops option
laptops=driver.find_element(By.XPATH, "/html/body/div[5]/div/div/div[4]/div[2]/div/div[1]/ul/li[4]/a")
laptops.click()

In [51]:
#create lists
name = []
price = []
OS = []
display = []
processor = []
HDD = []
RAM = []
weight = []
dimension = []
GPU = []

In [52]:
#Scraping Data
names=driver.find_elements(By.XPATH, "//span[@class='datahreflink']/h3")
for i in names:
    name.append(i.text)

time.sleep(1)

prices=driver.find_elements(By.XPATH, "//table[@id='summtable']//tr//td[3]")
for i in prices:
    price.append(i.text)    
    
time.sleep(1)

os=driver.find_elements(By.XPATH, "//div[@class='product-detail']/div/ul/li[1]/div/div")
for i in os:
    OS.append(i.text)

time.sleep(1)

displays=driver.find_elements(By.XPATH, "//div[@class='product-detail']/div/ul/li[2]/div/div")
for i in displays:
    display.append(i.text)

time.sleep(1)

processors=driver.find_elements(By.XPATH, "//div[@class='product-detail']/div/ul/li[4]/div/div")
for i in processors:
    processor.append(i.text)

time.sleep(1)

memory=driver.find_elements(By.XPATH, "//div[@class='product-detail']/div/ul/li[5]/div/div")# extrat HDD and RAM form xpath
for i in memory:
    HDD.append(i.text.split("/")[0])
    RAM.append(i.text.split("/")[1])

time.sleep(1)

weights=driver.find_elements(By.XPATH, "//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")# extrat weight form xpath
for i in weights:
    weight.append(i.text)

time.sleep(1)

dimension=[]
dimensions=driver.find_elements(By.XPATH, "//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]") 
for i in dimensions:
    dimension.append(i.text)

time.sleep(1)

GPUs=driver.find_elements(By.XPATH, "//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]") 
for i in GPUs:
    GPU.append(i.text)

time.sleep(1)

In [53]:
print(len(name), len(price), len(OS),len(display),len(processor),len(HDD),len(RAM),len(weight),len(dimension),len(GPU))

9 9 9 9 9 9 9 9 9 9


In [54]:
#create DataFrame
df=pd.DataFrame({"Name":name,
                "Price":price,
                "OS":OS,
                "Display":display,
                "Processor":processor,
                "HDD":HDD,
                "RAM":RAM,
                "Weight":weight,
                "Dimension":dimension,
                "Graphical processor":GPU})
df

,Name,Price,OS,Display,Processor,HDD,RAM,Weight,Dimension,Graphical processor
0,Acer Aspire 7 A715,"₹ 49,990",Windows 11 Home,"15.6"" (1920 x 1080)",AMD Ryzen 5-5500U | 2.1 GHz,512 GB SSD,16 GB DDR4,255 x 363 x 23 mm dimension & 2.15 kg weight,"₹ 49,990",4 GB DDR6 NVIDIA GeForce GTX 1650 Graphics card
1,MSI GF63 Thin,"₹ 52,990",Windows 10 Home,"15.6"" (1280 x 720)",11th Gen Intel Core i5-11400H | 4.50 GHz,512 GB SSD,8 GB DDR4,359 x 254 x 22 mm dimension & 1.86 kg weight,"₹ 52,990",4 GB DDR6 NVIDIA GeForce GTX1650 Graphics card
2,Acer Nitro 5 AN515-57,"₹ 64,350",Windows 11,"15.6"" (1920 x 1080)",11th Gen Intel Core i5-11400H | 2.2 GHz,512 GB SSD,16 GB DDR4,255 x 363 x 24 mm dimension & 2.2 kg weight,"₹ 64,350",4 GB DDR6 NVIDIA GeForce GTX 1650 Graphics card
3,Lenovo Legion 5,"₹ 74,990",Windows 10 Home,"15.6"" (1920 x 1080)",4th Gen AMD Ryzen 5-(4600H) | 3.0 GHz,1 TB HDD,8 GB DDR4,36.3 x 26 x 2.4 mm dimension & 2.3 kg weight,"₹ 74,990",NVIDIA GeForce GTX 1650 Graphics card
4,Lenovo Ideapad Gaming 3,"₹ 51,990",Windows 11 Home,"15.6"" (1920 x 1080)",11th Gen Intel Core i5-11300H | 3.1 GHz,512 GB SSD,8 GB DDR4,360 x 252 x 24 mm dimension & 2.25 kg weight,"₹ 51,990",4 GB DDR6 NVIDIA GeForce GTX 1650 Graphics card
5,HP Pavilion 15 (With Updated Specs),"₹ 84,990",Windows 11 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7-5800H | 4.4 GHz,512 GB SSD,16 GB DDR4,360 x 257 x 24 mm dimension & 1.98 kg weight,"₹ 84,990",4 GB DDR6 NVIDIA GeForce RTX 3050 Graphics card
6,HP Victus 16-D0354TX,"₹ 66,990",Windows 11 Home,"16.1"" (1920 x 1080)",11th Gen Intel Core i5-11400H | 4.5 GHz,512 GB SSD,8 GB DDR4,370 x 260 x 23.5 mm dimension & 2.46 kg weight,"₹ 66,990",4 GB DDR6 NVIDIA GeForce RTX 3050 Graphics card
7,Dell G15 511 Gaming Laptop,"₹ 67,690",Windows 11 Home,"15.6"" (1920 x 1080)",11th Gen Intel Core i5-11260H | 4.4 GHz,512 GB SSD,8 GB DDR4,272 x 357 x 27 mm dimension & 2.65 kg weight,"₹ 67,690",4 GB DDR6 NVIDIA GeForce RTX 3050 Graphics card
8,Acer Aspire 5 A515-57G,"₹ 59,990",Windows 11 Home,"15.6"" (1920 x 1080)",12th Gen Intel Core i5-1240P | 3.3 GHz,512 GB SSD,8 GB DDR4,238 x 363 x 18 mm dimension & 1.8 kg weight,"₹ 59,990",4 GB DDR6 NVIDIA GeForce RTX 2050 Graphics card


In [55]:
df.to_csv("Gaming_laptops.csv")

In [56]:
# Quit the browser
driver.quit()

Request:
7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped:
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [12]:
# Set up the driver
driver = webdriver.Chrome()

# Load the webpage
driver.get("http://www.forbes.com/")

In [19]:
#click hamburger link
button = driver.find_element(By.CLASS_NAME, "_69hVhdY4")
button.click()
                             
time.sleep(1)

#click billionaires link
button = driver.find_element(By.CLASS_NAME, "mpBfVZz3")
button.click()
                             
time.sleep(3)

#select world billioners 
world_billioners= driver.find_element(By.XPATH, "//li[@class='TjJgrPSg _2bNo56RE secondary']/a")
world_billioners.click()

time.sleep(4)

In [38]:
#create lists
rank = [] 
name = [] 
net_worth = [] 
age = []
country = [] 
source = []
industry = []

In [40]:
#Scraping Billonaire info

while(True):

    ranks= driver.find_elements(By.XPATH, "//div[@class='Table_rank___YBhk Table_dataCell__2QCve']")
    for i in ranks:
        rank.append(i.text)
    time.sleep(1)
    
    names= driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']//div[2]/div")
    for i in names:
        name.append(i.text)
    time.sleep(1)

    net_worths= driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']//div[3]/div")
    for i in net_worths:
        net_worth.append(i.text)
    time.sleep(1)     
    
    ages= driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']//div[4]/div")
    for i in ages:
        age.append(i.text)   
    time.sleep(1)
      
    countries= driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']//div[5]")
    for i in countries:
        country.append(i.text)
    time.sleep(1)
    
    sources= driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']//div[6]/div/span")
    for i in sources:
        source.append(i.text)    
    time.sleep(1)
        
    industries= driver.find_elements(By.XPATH, "//div[@class='TableRow_row__L-0Km']//div[7]/div")
    for i in industries:
        industry.append(i.text)    
    time.sleep(1)   
        
    try:
        next_button = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[3]/div[2]/div[2]/div[2]/div[2]/div[2]/div[1]/button[2]/div/span[1]")
        next_button.click()
    except:
        break  


In [44]:
time.sleep(2)        
#creating dataframe
df=pd.DataFrame({'Rank':rank[0:2626],
                'Name':name,
                'Net Worth':net_worth,
                'Age':age,
                'Country of Citizenship':country,
                'Source':source,
                'Industry':industry})

In [45]:
df

,Rank,Name,Net Worth,Age,Country of Citizenship,Source,Industry
0,1,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",Automotive
1,2,Jeff Bezos,$114 B,59,United States,Amazon,Technology
2,3,Larry Ellison,$107 B,78,United States,Oracle,Technology
3,4,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,Finance & Investments
4,5,Bill Gates,$104 B,67,United States,Microsoft,Technology
...,...,...,...,...,...,...,...
2621,2540,Yu Rong,$1 B,51,China,,
2622,2540,"Richard Yuengling, Jr.",$1 B,80,United States,,
2623,2540,Zhang Gongyun,$1 B,60,China,,
2624,2540,Zhang Guiping & family,$1 B,71,China,,


In [46]:
# Quit the browser
driver.quit()

Request:
8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video.

In [74]:
# Set up the driver
driver = webdriver.Chrome()

# Load the webpage
driver.get("https://www.youtube.com/")

In [75]:
#finding element for search bar
search_bar = driver.find_element(By.XPATH, '/html/body/ytd-app/div[1]/div/ytd-masthead/div[4]/div[2]/ytd-searchbox/form/div[1]/div[1]/input')
search_bar.send_keys("uptown funk") 
time.sleep(1)

#clicking on search button
search_btn = driver.find_element(By.ID, "search-icon-legacy")  
search_btn.click()
time.sleep(4)

#clicking on first video
link_click = driver.find_element(By.XPATH, "//a[@class='yt-simple-endpoint style-scope ytd-video-renderer']")
link_click.click()

In [60]:
# scroll to load comments
for _ in range(1000):
    driver.execute_script("window.scrollBy(0,5000)")

In [66]:
#create lists
comment = []
comment_time = []
up_vote = []

In [67]:
#scrape comments
comments = driver.find_elements(By.ID, "content-text")
for i in comments:
    if i.text is None:
        comment.append("--")
    else:
        comment.append(i.text)
time.sleep(5)

# scrape time when comment was posted
times = driver.find_elements(By.XPATH, "//a[contains(text(),'ago')]")
for i in times:
    comment_time.append(i.text)

for i in range(0,len(comment_time),2):
    comment_time.append(comment_time[i])
time.sleep(5)
    
# scrape the comment likes
up = driver.find_elements(By.XPATH, "//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for i in up:
    up_vote.append(i.text)

In [71]:
#create DataFrame
df = pd.DataFrame({'Comment':comment[0:500],
                'Comment Time':comment_time[0:500],
                'Up Vote':up_vote[0:500]})

In [72]:
df

,Comment,Comment Time,Up Vote
0,I can't explain how powerfully charming this s...,2 weeks ago (edited),
1,This song is timeless. I've been listening to ...,1 month ago,296
2,Bruno Mars sua presença na música mundial é um...,1 day ago,
3,"Let’s be honest, this song is still legendary....",6 days ago,994
4,Love the 80's vibe this has. Makes you want to...,2 weeks ago,
...,...,...,...
495,"""Smoother than a fresh jar of Skippy!"" is, wel...",13 days ago,9
496,My phone gets a little hot when I play this so...,3 weeks ago,
497,Remember when everyone talked about him like h...,2 years ago,2
498,Eu amo essa música,1 month ago,


In [73]:
# Quit the browser
driver.quit()

Request:
9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall
reviews, privates from price, dorms from price, facilities and property description.

In [137]:
# Set up the driver
driver = webdriver.Chrome()

# Load the webpage
driver.get("https://www.hostelworld.com/")

In [138]:
#locating the location search bar
search = driver.find_element(By.XPATH, "/html/body/div[3]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[1]/div/div[2]/input")
search.send_keys("London")

time.sleep(2)

#select london
london = driver.find_element(By.XPATH, '//*[@id="3"]/button/div[2]')
london.click()
time.sleep(2)

# do click in search button
search_btn = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/div[2]/div[2]/div/div/div/div[5]/button')
search_btn.click()

In [139]:
# create lists
name = []
distance = []
rate = []
total_reviews = []
overall_review = []
private = []
dorm = []
property_description = []
url = []

In [140]:
# Scrape data
while True:
    urls = driver.find_elements(By.XPATH, "//div[@class='property-card']/a")
    for u in urls:
        url.append(u.get_attribute("href"))

    time.sleep(1)

    names = driver.find_elements(By.XPATH, "//div[@class='property-name']")
    for n in names:
        name.append(n.text)

    time.sleep(1)

    try:
        dis = driver.find_elements(By.XPATH, '//*[@class="property-card"]/a/a/div[2]/div[3]')
        for d in dis:
            distance.append(d.text)
            time.sleep(1)
    except NoSuchElementException:
        distance.append('-')
        time.sleep(1)

    t_review = driver.find_elements(By.XPATH, '//*[@class="property-card"]/a/a/div[2]/div[2]/div/div[2]/span')
    for r in t_review:
        if r.text.strip() == '':
            dorm.append('-')
        else:
            dorm.append(r.text)
        time.sleep(1)

    try:
        review = driver.find_elements(By.XPATH, '//*[@class="property-card"]/a/a/div[2]/div[2]/div/div[1]/div[1]/span')
        for o_r in review:
            overall_review.append(o_r.text)
            time.sleep(1)
    except NoSuchElementException:
        overall_review.append('-')
        time.sleep(1)

    privates = driver.find_elements(By.XPATH, '//div[@class="property-card"]/a/a/div[2]/div[5]/div/div[1]')
    for p in privates:
        if p.text.strip() == '':
            private.append('-')
        else:
            private.append(p.text)
        time.sleep(1)

    dorms = driver.find_elements(By.XPATH, '//*[@class="property-card"]/a/a/div[2]/div[5]/div/div[2]/div[2]/strong')
    for do in dorms:
        if do.text.strip() == '':
            dorm.append('-')
        else:
            dorm.append(do.text)
        time.sleep(1)

    rates = driver.find_elements(By.XPATH, '//*[@class="property-card"]/a/a/div[2]/div[2]/div/div[1]/div[2]')
    for ra in rates:
        if ra.text.strip() == '':
            rate.append('-')
        else:
            rate.append(ra.text)
        time.sleep(1)

    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/div[2]/div[2]/div[1]/div/div/div/div/div[3]/div/button[2]')
        next_button.click()
        time.sleep(4)
    except:
        break

In [141]:
# Scrape property description
for ur in url:
    driver.get(ur)
    time.sleep(2)
    try:
        pd = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div[2]/section/div[6]/div/div[2]/div/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append('-')
    except:
        break

In [142]:
# Create DataFrame
df = pd.DataFrame({
    'Hostel Name': name,
    'Distance from city centre': distance,
    'Overall Review': overall_review,
    'Total Reviews': total_reviews,
    'Privates From Price': private,
    'Dorms From Price': dorm,
    'Rating': rate,
    'Property Description': property_description
})

AttributeError: 'WebElement' object has no attribute 'DataFrame'

In [145]:
df

,Hostel Name,Distance from city centre,Overall Review,Total Reviews,Privates From Price,Dorms From Price,Rating,Property Description


In [146]:
# Quit the browser
driver.quit()